<a href="https://colab.research.google.com/github/supportchelsea/Weather-Competition/blob/master/cf_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip3 install surprise

In [0]:
import surprise
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from tqdm import tqdm_notebook
import sys
import os
from matplotlib import pyplot as plt

%matplotlib inline

In [0]:
path = './gdrive/My Drive/GitHub/Weather Competition/'

In [0]:
hnb = pd.read_csv(path+'data/korea_hnb.csv').iloc[:,1:]

In [0]:
hnb.columns = list(i.split('.')[1] for i in hnb.columns)

In [0]:
# hnb.head()

,bor_nm,sale_dt,gen_cd,age_cd,category,qty
0,종로구,20160101,F,20~39,립컬러,73
1,종로구,20160101,F,20~39,립케어,219
2,종로구,20160101,F,20~39,마스크팩,657
3,종로구,20160101,F,20~39,바디로션,328
4,종로구,20160101,F,20~39,체중조절,109


In [0]:
segmentation = pd.DataFrame({'segmentation' : [hnb['bor_nm'][i]+' '+hnb['gen_cd'][i]+' '+hnb['age_cd'][i] for i in range(len(hnb))]})

In [0]:
df_hnb = pd.concat([hnb['sale_dt'],segmentation,hnb['category'],hnb['qty']], axis=1) 

In [0]:
# df_hnb.head()

In [0]:
rainy = [20180122.0,
 20180223.0,
 20180228.0,
 20180301.0,
 20180304.0,
 20180305.0,
 20180308.0,
 20180315.0,
 20180318.0,
 20180319.0,
 20180321.0,
 20180404.0,
 20180405.0,
 20180406.0,
 20180408.0,
 20180410.0,
 20180414.0,
 20180422.0,
 20180423.0,
 20180424.0,
 20180502.0,
 20180503.0,
 20180506.0,
 20180512.0,
 20180516.0,
 20180517.0,
 20180518.0,
 20180522.0,
 20180523.0,
 20180529.0,
 20180530.0,
 20180609.0,
 20180610.0,
 20180611.0,
 20180614.0,
 20180615.0,
 20180626.0,
 20180628.0,
 20180629.0,
 20180630.0,
 20180701.0,
 20180702.0,
 20180703.0,
 20180705.0,
 20180709.0,
 20180710.0,
 20180711.0,
 20180728.0,
 20180805.0,
 20180806.0,
 20180809.0,
 20180812.0,
 20180813.0,
 20180815.0,
 20180821.0,
 20180823.0,
 20180824.0,
 20180826.0,
 20180827.0,
 20180828.0,
 20180829.0,
 20180830.0,
 20180903.0,
 20180906.0,
 20180907.0,
 20180915.0,
 20180916.0,
 20180920.0,
 20180921.0,
 20180922.0,
 20181005.0,
 20181006.0,
 20181010.0,
 20181023.0,
 20181026.0,
 20181028.0,
 20181107.0,
 20181108.0,
 20181121.0,
 20181124.0,
 20181203.0,
 20181204.0]

In [0]:
rainy = [str(i).split('.')[0] for i in rainy]

In [0]:
rainy_date = []

for i in rainy:
  for j in df_hnb['sale_dt'] :
    if i == str(j):
      rainy_date.append(True)
    else:
      rainy_date.append(False)

In [0]:
# len(rainy_date)

In [0]:
df_rainy = pd.DataFrame(rainy_date)

In [0]:
df_rainy.columns = ['rainy_day']

In [0]:
hnb = pd.concat([df_hnb,df_rainy],axis=1)

In [0]:
hnb = hnb[hnb['sale_dt']>20171231]

In [0]:
rainy = hnb[hnb['rainy_day']==True].iloc[:,1:4].dropna() #비오는 날만

not_rainy = hnb[hnb['rainy_day']==False].iloc[:,1:4]

In [0]:
not_rainy.head(3)

,segmentation,category,qty
547246,종로구 F 20~39,네일,36.0
547247,종로구 F 20~39,립컬러,182.0
547248,종로구 F 20~39,립케어,146.0


In [0]:
len(rainy)

592

In [0]:
not_rainy.head(3)
len(not_rainy)

272436

In [0]:
rainy = [str(i).split('.')[0] for i in rainy]

In [0]:
l_qty = rainy['qty'].min()
u_qty = rainy['qty'].max()

print('Quantity range : {0} to {1}'.format(l_qty,u_qty))

Quantity range : 36.0 to 6567.0


In [0]:
reader = surprise.Reader()

In [0]:
surprise_rainy = surprise.Dataset.load_from_df(rainy,reader)
# df_tmp2 = surprise.Dataset.load_from_df(tmp2,reader)

In [0]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(surprise_rainy, test_size=0.25,shuffle=True,random_state=42)

In [0]:
sim_options = {'name': 'cosine',
               'user_based': False , # compute  similarities between items
               'min_support':2,
               'random_state':42
               }

algo_knn = surprise.KNNBasic(sim_options=sim_options)

# algo_SVDpp = surprise.SVDpp(user_based= False,random_state=42)

In [0]:
algo_knn.fit(trainset)
# algo_SVDpp.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
sim_knn = algo_knn.compute_similarities()
# sim_SVDpp = algo_SVDpp.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
# algo_SVDpp.sim_options

{'user_based': True}

In [0]:
knn_rainy = pd.DataFrame(sim_knn)
# SVDpp_rainy = pd.DataFrame(sim_SVDpp)

In [0]:
label = hnb['category'].unique()

In [0]:
knn_rainy.columns = label
knn_rainy.index  = label
knn_rainy

,네일,립컬러,립케어,마스크팩,바디로션,크림로션,훼이셜클렌저,선케어,체중조절,제모제
네일,1.000000,0.859883,0.912462,0.832488,0.903779,0.852938,0.688625,0.767230,0.781112,0.807020
립컬러,0.859883,1.000000,0.888123,0.906501,0.879121,0.805071,0.783195,0.874651,0.896595,0.608505
립케어,0.912462,0.888123,1.000000,0.904986,0.895806,0.852721,0.704597,0.552387,0.829305,0.656714
마스크팩,0.832488,0.906501,0.904986,1.000000,0.924282,0.860022,0.487415,0.853207,0.633229,0.546374
바디로션,0.903779,0.879121,0.895806,0.924282,1.000000,0.897915,0.782247,0.868015,0.773114,0.608161
크림로션,0.852938,0.805071,0.852721,0.860022,0.897915,1.000000,0.698607,0.956617,0.833076,0.616689
훼이셜클렌저,0.688625,0.783195,0.704597,0.487415,0.782247,0.698607,1.000000,0.980993,0.884968,0.865503
선케어,0.767230,0.874651,0.552387,0.853207,0.868015,0.956617,0.980993,1.000000,0.961068,0.982406
체중조절,0.781112,0.896595,0.829305,0.633229,0.773114,0.833076,0.884968,0.961068,1.000000,0.877449
제모제,0.807020,0.608505,0.656714,0.546374,0.608161,0.616689,0.865503,0.982406,0.877449,1.000000


In [0]:
# SVDpp.columns = label
# SVDpp.index = label
# SVDpp.

---
# Not Rainy

In [0]:
surprise_not_rainy = surprise.Dataset.load_from_df(not_rainy,reader)

trainset, testset = train_test_split(surprise_not_rainy, test_size=0.25,shuffle=True,random_state=42)

algo_knn.fit(trainset)

sim_knn = algo_knn.compute_similarities()

knn_not_rainy = pd.DataFrame(sim_knn)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
knn_not_rainy.columns = label
knn_not_rainy.index  = label
knn_not_rainy

,네일,립컬러,립케어,마스크팩,바디로션,크림로션,훼이셜클렌저,선케어,체중조절,제모제
네일,1.000000,0.665876,0.641221,0.633496,0.268096,0.568624,0.361723,0.602148,0.619832,0.625034
립컬러,0.665876,1.000000,0.702688,0.696861,0.272874,0.636336,0.376789,0.654483,0.679829,0.691434
립케어,0.641221,0.702688,1.000000,0.668609,0.263598,0.620537,0.353243,0.624909,0.660051,0.667010
마스크팩,0.633496,0.696861,0.668609,1.000000,0.324052,0.594840,0.433504,0.648138,0.656914,0.664077
바디로션,0.268096,0.272874,0.263598,0.324052,1.000000,0.251350,0.150086,0.240647,0.266883,0.282727
크림로션,0.568624,0.636336,0.620537,0.594840,0.251350,1.000000,0.336903,0.577517,0.592199,0.587775
훼이셜클렌저,0.361723,0.376789,0.353243,0.433504,0.150086,0.336903,1.000000,0.347526,0.365784,0.427517
선케어,0.602148,0.654483,0.624909,0.648138,0.240647,0.577517,0.347526,1.000000,0.620760,0.645575
체중조절,0.619832,0.679829,0.660051,0.656914,0.266883,0.592199,0.365784,0.620760,1.000000,0.652728
제모제,0.625034,0.691434,0.667010,0.664077,0.282727,0.587775,0.427517,0.645575,0.652728,1.000000


---
# Temperature

In [0]:
high_temp = [20180802.0, 20180801.0, 20180803.0, 20180731.0, 20180730.0, 20180815.0, 20180814.0, 20180729.0, 20180723.0, 20180724.0, 20180722.0, 20180804.0, 20180808.0, 20180727.0, 20180810.0, 20180807.0, 20180813.0, 20180811.0, 20180812.0, 20180806.0] 
low_temp = [20180203.0, 20181209.0, 20181229.0, 20180110.0, 20181230.0, 20181208.0, 20180129.0, 20180207.0, 20180205.0, 20180204.0, 20181227.0, 20180206.0, 20180112.0, 20181228.0, 20180127.0, 20180111.0, 20180123.0, 20180125.0, 20180124.0, 20180126.0]

---
# Humid

In [0]:
hnb = pd.read_csv(path+'data/korea_hnb.csv').iloc[:,1:]

In [0]:
upper_humid = [20180517.0, 20180701.0, 20180828.0, 20180702.0, 20181108.0, 20180516.0, 20180827.0, 20180315.0, 20180423.0, 20180512.0, 20181203.0, 20180921.0, 20180626.0, 20180824.0, 20180502.0, 20180710.0, 20180405.0, 20180829.0, 20180304.0, 20180506.0] 
lower_humid = [20180128.0, 20181128.0, 20181226.0, 20180205.0, 20180206.0, 20180412.0, 20180126.0, 20181228.0, 20181208.0, 20181207.0, 20181209.0, 20180125.0, 20180221.0, 20180218.0, 20180124.0, 20181229.0, 20180215.0, 20180413.0, 20180217.0, 20181227.0]